In [24]:
import yfinance as yf
import pandas as pd
import numpy as np

import scipy

### Utility Functions

In [21]:
def get_drift_volatility(symbol, period='45d'):
    
    # Get NSE Historic Data
    ticker = yf.Ticker(symbol)
    df = ticker.history(period=period)['Close'].reset_index()

    # Calculate Daily Returns
    df['Prev Close'] = df['Close'].shift(1)
    df['Daily Rt'] = np.log(df['Close'] / df['Prev Close'])

    # Calculate Daily and Annualized Drift
    daily_drift = df['Daily Rt'].mean()
    annual_drift = daily_drift * 252

    # Calculate Daily and Annualized Volatility
    daily_volatility = df['Daily Rt'].std()
    annual_volatility = daily_volatility * np.sqrt(252)

    # Drift Mean
    drift_mean = daily_drift - (0.5 * (daily_volatility ** 2))

    return daily_drift, annual_drift, daily_volatility, annual_volatility, drift_mean

In [56]:
def gbm(initial_shock, drift_mean, daily_volatility, t=5):

    price = []

    for i in range(t):
        z = scipy.stats.norm.ppf(np.random.rand())
        log_return = drift_mean + daily_volatility * z
        p = initial_shock * np.exp(log_return)
        initial_shock = p
        price.append(p)

    return price

### Experiment

In [22]:
# URL to get the list of NSE stocks
nse_tickers_url = '../data/static/ind_nifty50list.csv'

# Read the data from the URL
tickers_df = pd.read_csv(nse_tickers_url)

# Extract the Symbol column
tickers = tickers_df['Symbol'].tolist()

# Print all NSE tickers
vol_df = []
for ticker in tickers:
   dd, ad, dv, av, dm = get_drift_volatility(ticker+'.NS')
   tmp = {
      'Symbol': ticker,
      'Daily Drift': dd,
      'Annual Drift': ad,
      'Daily Volatility': dv,
      'Annual Volatility': av,
      'Drift Mean': dm
   }
   vol_df.append(tmp)

vol_df = pd.DataFrame(vol_df)

In [23]:
vol_df

,Symbol,Daily Drift,Annual Drift,Daily Volatility,Annual Volatility,Drift Mean
0,ADANIENT,-0.001038,-0.261539,0.021944,0.348346,-0.001279
1,ADANIPORTS,0.000300,0.075661,0.019141,0.303857,0.000117
2,APOLLOHOSP,-0.000913,-0.230025,0.016252,0.257998,-0.001045
3,ASIANPAINT,-0.000463,-0.116738,0.014177,0.225048,-0.000564
4,AXISBANK,0.000733,0.184669,0.014272,0.226566,0.000631
5,BAJAJ-AUTO,0.000381,0.095919,0.014903,0.236585,0.000270
6,BAJFINANCE,0.001030,0.259582,0.017784,0.282305,0.000872
7,BAJAJFINSV,-0.000124,-0.031261,0.012840,0.203825,-0.000206
8,BPCL,0.000078,0.019633,0.022513,0.357383,-0.000176
9,BHARTIARTL,0.002680,0.675474,0.014560,0.231133,0.002574


### Rough

In [79]:
get_volatility('ZYDUSLIFE.NS')

(0.020992579894727276, 0.33324687467461717)

In [3]:
ticker = yf.Ticker('ZYDUSLIFE.NS')

In [4]:
df = ticker.history(period='45d')['Close'].reset_index()

In [5]:
df['Prev Close'] = df['Close'].shift(1)

In [6]:
df['Daily Rt'] = np.log(df['Close'] / df['Prev Close'])

In [72]:
df.to_csv('../data/test/zyduslife.csv', index=False)

In [7]:
daily_volatility = df['Daily Rt'].std()
daily_volatility

0.020992579894727276

In [17]:
annual_volatility = daily_volatility * 252
annual_volatility

5.290130133471274

In [10]:
daily_drift = df['Daily Rt'].mean()
daily_drift

0.0023213346431457697

In [11]:
annual_drift = daily_drift * np.sqrt(252)
annual_drift

0.036850045053135445

In [47]:
drift_mean = daily_drift * (0.5 * (daily_volatility**2))
drift_mean

5.114926372216871e-07

In [75]:
df.shape

(46, 4)

In [41]:
print(scipy.stats.norm.ppf(0.001))
for i in np.arange(0.1, 1, 0.1):
    print(scipy.stats.norm.ppf(i))

scipy.stats.norm.ppf(0.99)

-3.090232306167813
-1.2815515655446004
-0.8416212335729142
-0.5244005127080407
-0.2533471031357997
0.0
0.2533471031357997
0.524400512708041
0.8416212335729143
1.2815515655446004


2.3263478740408408

In [52]:
z = scipy.stats.norm.ppf(np.random.rand())
z

-0.22154311410959399

In [53]:
log_return = drift_mean + daily_volatility * z
log_return

-0.004650250030435112

In [50]:
# df

In [55]:
1103 * np.exp(log_return)

1097.8826818426446

In [59]:
for t in range(5):
    print(gbm(1103, drift_mean, daily_volatility))

[1107.6775052973544, 1116.2217552350326, 1096.8689260428166, 1073.9912096456549, 1067.7138614775863]
[1137.8366507936128, 1123.0422409478956, 1113.93854936939, 1107.9055468796291, 1111.2749651439062]
[1115.6323054216477, 1080.4382072839521, 1119.3698838453004, 1081.2854003543848, 1113.4145764610294]
[1101.7925211188287, 1126.6740808010804, 1114.0982832161392, 1141.237050817523, 1154.2829107776654]
[1098.487855504138, 1129.1833658466135, 1128.5694668456322, 1141.6288328752357, 1139.8646827914936]
